In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf

# 损失函数losses和regularizers
一般来说，监督学习的目标函数由损失函数和正则化项组成。（Objective = Loss + Regularization）

对于keras模型，目标函数中的正则化项一般在各层中指定，例如使用Dense的 kernel_regularizer 和 bias_regularizer等参数指定权重使用l1或者l2正则化项，此外还可以用kernel_constraint 和 bias_constraint等参数约束权重的取值范围，这也是一种正则化手段。

损失函数在模型编译时候指定。对于回归模型，通常使用的损失函数是均方损失函数 mean_squared_error。

对于二分类模型，通常使用的是二元交叉熵损失函数 binary_crossentropy。

对于多分类模型，如果label是one-hot编码的，则使用类别交叉熵损失函数 categorical_crossentropy。如果label是类别序号编码的，则需要使用稀疏类别交叉熵损失函数 sparse_categorical_crossentropy。

如果有需要，也可以自定义损失函数，自定义损失函数需要接收两个张量y_true,y_pred作为输入参数，并输出一个标量作为损失函数值。


In [6]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential()
model.add(
    tf.keras.Input(shape=(None,64))
)
model.add(tf.keras.layers.Dense(
    units=10,
    kernel_regularizer=tf.keras.regularizers.L2(0.01),
    activity_regularizer = tf.keras.regularizers.L1(0.01),
    kernel_constraint = tf.keras.constraints.MaxNorm(max_value=2,axis=0)
))
model.add(tf.keras.layers.Dense(5,kernel_regularizer=tf.keras.regularizers.L1L2(0.01, 0.01),activation="sigmoid"))
model.compile(optimizer="rmsprop", loss="mse", metrics=["AUC"])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, None, 10)       │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 5)        │            55 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 705 (2.75 KB)

 Trainable params: 705 (2.75 KB)

 Non-trainable params: 0 (0.00 B)

## 内置损失函数
内置的损失函数一般有类的实现和函数的实现两种形式。
* mse,MSE 
* mae,MAE,
* mape,MAPE, mean_absolute_percentage_error

In [7]:

y_true = np.random.randint(0,2,(2,3))
y_pred = np.random.random(size=(2, 3))
tf.keras.losses.mean_squared_logarithmic_error(y_true, y_pred)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.11906506, 0.17642627])>

## binary_crossentropy

In [8]:
y_true = [[0,1],[0,0]]
y_pred = [[0.6, 0.4], [0.4, 0.6]]
loss = tf.keras.losses.binary_crossentropy(y_true,y_pred)
loss

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.91629076, 0.7135582 ], dtype=float32)>

In [9]:
target=[0,1]
output = [0.6, 0.4]
target = np.array(target)
output = np.array(output)
output = np.clip(output, 1e-07, 1.0 - 1e-07)
bce = target * np.log(output)
bce += (1.0 - target) * np.log(1.0 - output)
bce = -bce
np.mean(bce, axis=-1)

0.916290731874155

In [10]:
-(np.log(1-0.4)+np.log(1-0.6))/2

0.7135581778200728

In [11]:
-(np.log(1-0.6)+ np.log(0.4))/2

0.916290731874155

## categorical_crossentropy
```
def categorical_crossentropy(
    y_true, y_pred, from_logits=False, label_smoothing=0.0, axis=-1
):
```

In [12]:
y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
loss

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.05129331, 2.3025851 ], dtype=float32)>

In [13]:
-(np.log(0.95))/1

0.05129329438755058

In [14]:
-(np.log(0.1))

2.3025850929940455

In [15]:
y_true = np.random.choice([-1, 1], size=(2, 3))
y_pred = np.random.random(size=(2, 3))
tf.keras.losses.hinge(y_true, y_pred)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([1.13306519, 0.4088818 ])>

In [16]:
y_true

array([[-1,  1, -1],
       [ 1,  1,  1]])

## categorical_hinge

In [59]:
y_true = np.random.randint(0, 3, size=(2,),dtype="int")
y_true

array([1, 1])

In [60]:
array = np.eye(3)
array

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [61]:
y_true = array[y_true]
y_true

array([[0., 1., 0.],
       [0., 1., 0.]])

In [62]:
y_pred = np.random.random(size=(2, 3))
y_pred

array([[0.08490878, 0.6943042 , 0.92105105],
       [0.0476161 , 0.73578765, 0.54121682]])

In [63]:
tf.keras.losses.categorical_hinge(y_true, y_pred)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([1.22674685, 0.80542917])>